## Import pandas

In [1]:
import pandas as pd

## Import CMS Benefits_Cost_Sharing_2023

In [22]:
cms_data_1 = pd.read_csv(r'\Users\burke\Downloads/Benefits_Cost_Sharing_PUF.csv', low_memory=False)

In [24]:
print(cms_data_1.head())

   BusinessYear StateCode  IssuerId SourceName           ImportDate  \
0          2023        AK     21989       HIOS  2022-11-19 01:01:13   
1          2023        AK     21989       HIOS  2022-11-19 01:01:13   
2          2023        AK     21989       HIOS  2022-11-19 01:01:13   
3          2023        AK     21989       HIOS  2022-11-19 01:01:13   
4          2023        AK     21989       HIOS  2022-11-19 01:01:13   

  StandardComponentId             PlanId                      BenefitName  \
0      21989AK0030001  21989AK0030001-00  Routine Dental Services (Adult)   
1      21989AK0030001  21989AK0030001-00     Dental Check-Up for Children   
2      21989AK0030001  21989AK0030001-00        Basic Dental Care - Child   
3      21989AK0030001  21989AK0030001-00              Orthodontia - Child   
4      21989AK0030001  21989AK0030001-00        Major Dental Care - Child   

    CopayInnTier1 CopayInnTier2  ... IsEHB IsCovered QuantLimitOnSvc LimitQty  \
0  Not Applicable           N

In [25]:
print(cms_data_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219663 entries, 0 to 2219662
Data columns (total 24 columns):
 #   Column               Dtype  
---  ------               -----  
 0   BusinessYear         int64  
 1   StateCode            object 
 2   IssuerId             int64  
 3   SourceName           object 
 4   ImportDate           object 
 5   StandardComponentId  object 
 6   PlanId               object 
 7   BenefitName          object 
 8   CopayInnTier1        object 
 9   CopayInnTier2        object 
 10  CopayOutofNet        object 
 11  CoinsInnTier1        object 
 12  CoinsInnTier2        object 
 13  CoinsOutofNet        object 
 14  IsEHB                object 
 15  IsCovered            object 
 16  QuantLimitOnSvc      object 
 17  LimitQty             float64
 18  LimitUnit            object 
 19  Exclusions           object 
 20  Explanation          object 
 21  EHBVarReason         object 
 22  IsExclFromInnMOOP    object 
 23  IsExclFromOonMOOP    object 
dty

In [26]:
cms_data_1.duplicated().sum()

0

In [27]:
cms_data_1.isnull().sum()

BusinessYear                 0
StateCode                    0
IssuerId                     0
SourceName                   0
ImportDate                   0
StandardComponentId          0
PlanId                       0
BenefitName                  0
CopayInnTier1           503349
CopayInnTier2          1852011
CopayOutofNet           503349
CoinsInnTier1           503349
CoinsInnTier2          1852011
CoinsOutofNet           503349
IsEHB                   578607
IsCovered               126678
QuantLimitOnSvc        1612945
LimitQty               1924276
LimitUnit              1924243
Exclusions             2082783
Explanation            1536817
EHBVarReason           1674567
IsExclFromInnMOOP       390070
IsExclFromOonMOOP       389650
dtype: int64

## Number of Plans by State

In [ ]:
State_Plans = cms_data_1.groupby('StateCode')['PlanId'].sum()

In [ ]:
State_Plans.head()

In [ ]:
State_Plans.sort_index()